## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key = g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Shimoda,JP,79.16,broken clouds,34.6667,138.9500,Shimoda Tokyu Hotel
1,1,Jamestown,US,83.79,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
2,2,Faanui,PF,78.04,clear sky,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
3,3,Mahon,ES,75.76,clear sky,39.8885,4.2658,ARTIEM Capri
4,4,Ponnani,IN,74.52,overcast clouds,10.7670,75.9252,Rouba Residency


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the lowest temperature you would like for your trip? "))
max_temp = float(input("What is the highest temperature you would like for your trip? "))

What is the lowest temperature you would like for your trip? 65
What is the highest temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
tempfilter_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                            (city_data_df["Max Temp"] >= min_temp)]
tempfilter_city_data_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Shimoda,JP,79.16,broken clouds,34.6667,138.9500,Shimoda Tokyu Hotel
1,1,Jamestown,US,83.79,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
2,2,Faanui,PF,78.04,clear sky,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
3,3,Mahon,ES,75.76,clear sky,39.8885,4.2658,ARTIEM Capri
4,4,Ponnani,IN,74.52,overcast clouds,10.7670,75.9252,Rouba Residency
5,5,Ponta Do Sol,PT,71.71,clear sky,32.6667,-17.1000,Hotel do Campo
6,6,Hithadhoo,MV,80.42,overcast clouds,-0.6000,73.0833,Scoop Guest House
7,7,Mantua,IT,81.64,clear sky,45.1667,10.7833,Hotel la Favorita
8,8,Carnarvon,AU,72.21,scattered clouds,-24.8667,113.6333,Hospitality Carnarvon
9,9,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,Paradise Inn


In [5]:
# 4a. Determine if there are any empty rows.
print(f"Null\n{tempfilter_city_data_df.isnull().sum()}\n")
print(f"Not Null\n{tempfilter_city_data_df.notnull().sum()}")

Null
City_ID                0
City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

Not Null
City_ID                279
City                   279
Country                279
Max Temp               279
Current Description    279
Lat                    279
Lng                    279
Hotel Name             279
dtype: int64


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df = tempfilter_city_data_df.dropna()
print(f"Null\n{clean_cities_df.isnull().sum()}\n")
print(f"Not Null\n{clean_cities_df.notnull().sum()}")

Null
City_ID                0
City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

Not Null
City_ID                279
City                   279
Country                279
Max Temp               279
Current Description    279
Lat                    279
Lng                    279
Hotel Name             279
dtype: int64


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Shimoda,JP,79.16,broken clouds,34.6667,138.9500,
1,Jamestown,US,83.79,clear sky,42.0970,-79.2353,
2,Faanui,PF,78.04,clear sky,-16.4833,-151.7500,
3,Mahon,ES,75.76,clear sky,39.8885,4.2658,
4,Ponnani,IN,74.52,overcast clouds,10.7670,75.9252,
5,Ponta Do Sol,PT,71.71,clear sky,32.6667,-17.1000,
6,Hithadhoo,MV,80.42,overcast clouds,-0.6000,73.0833,
7,Mantua,IT,81.64,clear sky,45.1667,10.7833,
8,Carnarvon,AU,72.21,scattered clouds,-24.8667,113.6333,
9,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
print(clean_hotel_df.count())
clean_hotel_df.head(10)

City                   279
Country                279
Max Temp               279
Current Description    279
Lat                    279
Lng                    279
Hotel Name             279
dtype: int64


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Shimoda,JP,79.16,broken clouds,34.6667,138.9500,Shimoda Tokyu Hotel
1,Jamestown,US,83.79,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
2,Faanui,PF,78.04,clear sky,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
3,Mahon,ES,75.76,clear sky,39.8885,4.2658,ARTIEM Capri
4,Ponnani,IN,74.52,overcast clouds,10.7670,75.9252,Rouba Residency
5,Ponta Do Sol,PT,71.71,clear sky,32.6667,-17.1000,Hotel do Campo
6,Hithadhoo,MV,80.42,overcast clouds,-0.6000,73.0833,Scoop Guest House
7,Mantua,IT,81.64,clear sky,45.1667,10.7833,Hotel la Favorita
8,Carnarvon,AU,72.21,scattered clouds,-24.8667,113.6333,Hospitality Carnarvon
9,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,Paradise Inn


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Weather_Database/WeatherPy_Database.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(33.31,75.76), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))